In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from statsmodels.tools.tools import add_constant
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import make_pipeline

https://towardsdatascience.com/support-vector-regression-svr-one-of-the-most-flexible-yet-robust-prediction-algorithms-4d25fbdaca60/

In [2]:
DROPPED = [
    "dist_360_SPEED", "dist_360_THROTTLE", "dist_360_STEER", "dist_360_BRAKE",
    "dist_360_CURRENTLAPTIMEINMS", "dist_360_LAPDISTANCE", "dist_360_WORLDPOSITIONX", "dist_360_WORLDPOSITIONY",
    "dist_360_WORLDFORWARDDIRX", "dist_360_WORLDFORWARDDIRY", "dist_360_YAW", "dist_360_PITCH",
    "dist_360_ROLL", "dist_360_left_dist", "dist_360_right_dist", "dist_360_dist_apex_1",
    "dist_360_dist_apex_2", "dist_360_angle_to_apex1", "dist_360_angle_to_apex2", "dist_360_proj_from_ref",
    "dist_430_SPEED", "dist_430_THROTTLE", "dist_430_STEER", "dist_430_BRAKE",
    "dist_430_CURRENTLAPTIMEINMS", "dist_430_LAPDISTANCE", "dist_430_WORLDPOSITIONX", "dist_430_WORLDPOSITIONY",
    "dist_430_WORLDFORWARDDIRX", "dist_430_WORLDFORWARDDIRY", "dist_430_YAW", "dist_430_PITCH",
    "dist_430_ROLL", "dist_430_left_dist", "dist_430_right_dist", "dist_430_dist_apex_1",
    "dist_430_dist_apex_2", "dist_430_angle_to_apex1", "dist_430_angle_to_apex2", "dist_430_proj_from_ref",
    "dist_530_SPEED", "dist_530_THROTTLE", "dist_530_STEER", "dist_530_BRAKE",
    "dist_530_CURRENTLAPTIMEINMS", "dist_530_LAPDISTANCE", "dist_530_WORLDPOSITIONX", "dist_530_WORLDPOSITIONY",
    "dist_530_WORLDFORWARDDIRX", "dist_530_WORLDFORWARDDIRY", "dist_530_YAW", "dist_530_PITCH",
    "dist_530_ROLL", "dist_530_left_dist", "dist_530_right_dist", "dist_530_dist_apex_1",
    "dist_530_dist_apex_2", "dist_530_angle_to_apex1", "dist_530_angle_to_apex2", "dist_530_proj_from_ref",
    "BPS_right_dist", "BPE_right_dist", "THS_right_dist", "THE_right_dist", "STS_right_dist",
    "STM_right_dist", "STE_right_dist", "APX1_right_dist", "APX2_right_dist", "BPS_CURRENTLAPTIMEINMS",
    "BPE_CURRENTLAPTIMEINMS", "THS_CURRENTLAPTIMEINMS", "THE_CURRENTLAPTIMEINMS", "STS_CURRENTLAPTIMEINMS",
    "STM_CURRENTLAPTIMEINMS", "STE_CURRENTLAPTIMEINMS", "APX1_CURRENTLAPTIMEINMS", "APX2_CURRENTLAPTIMEINMS"
]

UNION = ["lap_id", "invalid_lap", 'APX1_BRAKE', 'APX1_SPEED', 'APX1_STEER', 'APX1_WORLDFORWARDDIRX', 'APX1_WORLDPOSITIONX', 'APX1_WORLDPOSITIONY',
         'APX1_YAW', 'APX1_angle_to_apex2', 'APX1_dist_apex_1', 'APX1_left_dist', 'APX1_proj_from_ref', 'APX2_PITCH',
         'APX2_SPEED', 'APX2_STEER', 'APX2_THROTTLE', 'APX2_WORLDFORWARDDIRY', 'APX2_WORLDPOSITIONY', 'APX2_YAW',
         'APX2_angle_to_apex1', 'APX2_angle_to_apex2', 'APX2_dist_apex_1', 'BPE_LAPDISTANCE', 'BPE_ROLL', 'BPE_SPEED',
         'BPE_STEER', 'BPE_WORLDFORWARDDIRX', 'BPE_WORLDFORWARDDIRY', 'BPE_YAW', 'BPE_angle_to_apex1', 'BPE_angle_to_apex2',
         'BPE_dist_apex_1', 'BPE_ext_TIMETOINMS', 'BPE_left_dist', 'BPE_proj_from_ref', 'BPS_PITCH', 'BPS_ROLL',
         'BPS_SPEED', 'BPS_STEER', 'BPS_THROTTLE', 'BPS_WORLDPOSITIONX', 'BPS_YAW', 'BPS_angle_to_apex1',
         'BPS_ext_TIMETOINMS', 'BPS_left_dist', 'STE_LAPDISTANCE', 'STE_PITCH', 'STE_ROLL', 'STE_SPEED',
         'STE_STEER', 'STE_THROTTLE', 'THS_WORLDFORWARDDIRX', 'STE_WORLDFORWARDDIRX',
         'STE_WORLDFORWARDDIRY', 'STE_YAW', 'STE_angle_to_apex1', 'STE_angle_to_apex2',
         'STE_dist_apex_1', 'STE_ext_LAPDISTANCE', 'STE_ext_TIMETOINMS', 'STE_proj_from_ref', 'STM_BRAKE',
         'STM_LAPDISTANCE', 'BPS_left_dist', 'STM_ROLL', 'STM_SPEED', 'STM_STEER', 'STM_THROTTLE',
         'STM_THROTTLE', 'STE_LAPDISTANCE', 'STM_WORLDFORWARDDIRX', 'STM_WORLDFORWARDDIRX', 'STE_YAW', 'STM_WORLDFORWARDDIRY',
         'STM_WORLDPOSITIONX', 'THS_PITCH', 'STM_WORLDPOSITIONY', 'STM_YAW', 'STM_angle_to_apex1',
         'STM_angle_to_apex2', 'STE_angle_to_apex2', 'STM_dist_apex_1', 'STM_left_dist', 'STS_BRAKE', 'STS_LAPDISTANCE',
         'STS_PITCH', 'STS_SPEED', 'STS_STEER', 'STS_THROTTLE', 'STS_WORLDFORWARDDIRX', 'STS_WORLDFORWARDDIRY',
         'STS_WORLDPOSITIONX', 'STS_WORLDPOSITIONY', 'STS_YAW', 'APX2_YAW', 'STS_angle_to_apex1', 'STS_angle_to_apex2',
         'STS_dist_apex_2', 'STS_ext_LAPDISTANCE', 'STS_ext_TIMETOINMS', 'APX1_WORLDPOSITIONX',
         'STS_left_dist', 'STS_proj_from_ref', 'THE_BRAKE', 'THE_LAPDISTANCE', 'THE_ROLL', 'THE_SPEED', 'THE_WORLDFORWARDDIRX',
         'THE_YAW', 'THE_dist_apex_1', 'THE_dist_apex_2', 'THE_ext_LAPDISTANCE', 'THE_ext_TIMETOINMS', 'THE_proj_from_ref',
         'THS_PITCH', 'THS_ROLL', 'THS_SPEED', 'THS_SPEED', 'APX1_left_dist', 'THS_STEER', 'THS_THROTTLE', 'THS_WORLDFORWARDDIRX',
         'THS_WORLDFORWARDDIRY', 'THS_WORLDPOSITIONY', 'THS_YAW', 'THS_ext_LAPDISTANCE', 'THS_ext_TIMETOINMS', 'THS_proj_from_ref'
]

TARGET = ["Target_CURRENTLAPTIMEINMS"]

selected_features = [
    "APX1_BRAKE", "APX1_SPEED", "APX1_STEER", "APX1_WORLDFORWARDDIRX",
    "APX1_YAW", "APX1_angle_to_apex2", "APX1_proj_from_ref", "APX2_SPEED",
    "APX2_STEER", "APX2_THROTTLE", "APX2_WORLDPOSITIONY", "APX2_angle_to_apex1",
    "APX2_angle_to_apex2", "APX2_dist_apex_1", "BPE_ROLL", "BPE_STEER",
    "BPE_WORLDFORWARDDIRY", "BPE_YAW", "BPE_angle_to_apex1", "BPE_angle_to_apex2",
    "BPE_ext_TIMETOINMS", "BPE_left_dist", "BPE_proj_from_ref", "BPS_PITCH",
    "BPS_ROLL", "BPS_STEER", "BPS_THROTTLE", "BPS_YAW",
    "BPS_angle_to_apex1", "BPS_ext_TIMETOINMS", "STE_ROLL", "STE_STEER",
    "STE_THROTTLE", "STE_angle_to_apex1", "STE_ext_LAPDISTANCE", "STE_ext_TIMETOINMS",
    "STE_proj_from_ref", "STM_BRAKE", "STM_ROLL", "STM_SPEED",
    "STM_STEER", "STM_WORLDFORWARDDIRY", "STM_YAW", "STM_angle_to_apex1",
    "STM_angle_to_apex2", "STM_left_dist", "STS_BRAKE", "STS_STEER",
    "STS_THROTTLE", "STS_angle_to_apex1", "STS_angle_to_apex2", "STS_ext_TIMETOINMS",
    "STS_proj_from_ref", "THE_BRAKE", "THE_ROLL", "THE_SPEED",
    "THE_WORLDFORWARDDIRX", "THE_YAW", "THE_dist_apex_1", "THE_proj_from_ref",
    "THS_ROLL", "THS_STEER", "THS_THROTTLE", "THS_YAW",
    "THS_proj_from_ref"
]

another_selected_features = [
    "APX2_SPEED", "STM_SPEED", "STE_ext_TIMETOINMS", "STS_angle_to_apex1",
    "APX2_STEER", "THS_proj_from_ref", "STE_THROTTLE", "APX1_SPEED",
    "STM_angle_to_apex2", "STE_proj_from_ref", "THE_ROLL", "STE_ext_LAPDISTANCE",
    "STS_ext_TIMETOINMS", "THE_SPEED", "BPS_angle_to_apex1", "STM_YAW",
    "STE_STEER", "APX1_YAW", "APX1_WORLDFORWARDDIRX", "APX2_THROTTLE",
    "APX1_proj_from_ref", "STE_angle_to_apex1", "BPS_ROLL", "STS_BRAKE",
    "THE_YAW", "STE_ROLL", "STS_proj_from_ref", "THE_WORLDFORWARDDIRX",
    "BPE_angle_to_apex1", "BPE_WORLDFORWARDDIRY", "APX2_angle_to_apex1", "THS_ROLL",
    "BPE_proj_from_ref", "THS_THROTTLE", "BPS_PITCH", "BPS_THROTTLE",
    "STS_angle_to_apex2", "THE_proj_from_ref", "APX1_BRAKE", "BPE_angle_to_apex2",
    "THS_YAW", "BPE_YAW", "APX2_angle_to_apex2", "BPE_ROLL",
    "BPS_YAW", "BPE_ext_TIMETOINMS", "APX1_STEER", "APX1_angle_to_apex2",
    "BPS_STEER", "STM_angle_to_apex1", "STM_left_dist"
]



# Uploading Data and removing outliers and features

In [10]:
data = pd.read_csv("final_data_product.csv")
data = data.dropna().drop_duplicates().drop(columns=DROPPED)
target_mean = data["Target_CURRENTLAPTIMEINMS"].mean()
target_std = data["Target_CURRENTLAPTIMEINMS"].std()
data = data[data['Target_CURRENTLAPTIMEINMS'] < target_mean + 3 * target_std] # removes 12 longest times
y = data["Target_CURRENTLAPTIMEINMS"]
X = data.drop(columns=["Target_CURRENTLAPTIMEINMS", "lap_id", "invalid_lap"])

target_columns = [
    'target_CURRENTLAPTIMEINMS', '_LAPDISTANCE', '_WORLDPOSITIONX', 
    '_WORLDPOSITIONY', '_STEER', '_BRAKE', '_THROTTLE'#, '_SPEED',
]

selected_columns = [col for col in X.columns if col.endswith(tuple(target_columns))]

X = X[selected_columns]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=42)

scaler_X = StandardScaler()
scaler_X_split = StandardScaler()
scaler_y = StandardScaler()
scaler_y_split = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
X_train_scaled = scaler_X_split.fit_transform(X_train)
X_test_scaled = scaler_X_split.transform(X_test)

y_train_scaled = scaler_y_split.fit_transform(y_train.to_numpy().reshape(-1, 1)).ravel()
y_test_scaled = scaler_y_split.transform(y_test.to_numpy().reshape(-1, 1)).ravel()
y_scaled = scaler_y.fit_transform(y.to_numpy().reshape(-1, 1)).ravel()

# Feature selection

### Mutual information

In [ ]:
from sklearn.feature_selection import mutual_info_regression

m_info = mutual_info_regression(X, y)
Scores = pd.DataFrame(sorted(zip(X.columns, m_info), key=lambda x: x[1], reverse=True), columns=["feature", "mi_score"])
pd.set_option('display.max_rows', 200)
Scores

# Detecting mutlicollinearity

### Variance inflation factor

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
X = add_constant(X)
vif_data['feature'] = X.columns
vif_data["VIF"] = [round(variance_inflation_factor(X.values, i), 4) for i in range(X.shape[1])]
vif_data[vif_data["VIF"] < 10].sort_values(by="VIF")#.iloc[:,0]
# vif_data

### SVR (scaled data without feature selection and without addressing mutlicollinearity)

In [5]:
poly_parameters = {
    'kernel': ['poly'],
    'degree': [3, 4, 5, 6, 7, 8],
    'gamma': ['scale', 'auto'],
    'coef0': [1, 3, 5, 7],
    'tol': [1e-3],
    'C': [0.1, 0.2, 0.3, 0.4, 0.5],
    'epsilon': [0.0005],
    'shrinking': [True],
    'verbose': [True],
    'max_iter': [-1]
}

from sklearn.svm import SVR

grid_poly = GridSearchCV(
    estimator=SVR(),
    param_grid=poly_parameters,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=2
)

grid_poly.fit(X_train_scaled, y_train_scaled)
print("Best parameters:", grid_poly.best_params_)
print("Best RMSE:", abs(grid_poly.best_score_))

Fitting 5 folds for each of 240 candidates, totalling 1200 fits
[LibSVM]Best parameters: {'C': 0.1, 'coef0': 3, 'degree': 3, 'epsilon': 0.0005, 'gamma': 'scale', 'kernel': 'poly', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': True}
Best RMSE: 0.5256923169592099


Fitting 5 folds for each of 2940 candidates, totalling 14700 fits
[LibSVM]Best parameters: {'C': 0.1, 'coef0': 7, 'degree': 5, 'epsilon': 0.0001, 'gamma': 'auto', 'kernel': 'poly', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': True}
Best RMSE: 2507.1384803500814

In [38]:
y_pred_poly_scaled = grid_poly.predict(X_test_scaled)
y_pred_poly = scaler_y_split.inverse_transform(y_pred_poly_scaled.reshape(-1,1))

rmse_poly = root_mean_squared_error(y_pred_poly, y_test)
r2_poly = r2_score(y_pred_poly, y_test)
print("RMSE for poly kernel:", rmse_poly)
print("R² for poly kernel:", r2_poly)

RMSE for poly kernel: 1567.4898718441304
R² for poly kernel: 0.6870556241692698


In [44]:
cv = KFold(n_splits=3, shuffle=True, random_state=42)
rmse_cv_poly = cross_val_score(grid_poly, X, y,
                               scoring="neg_root_mean_squared_error",
                               cv=cv, n_jobs=-1, verbose=3)

print("\n5-fold CV RMSE:")
print("Fold RMSEs:", np.round(-rmse_cv_poly, 3))
print("Mean RMSE :", np.round(-rmse_cv_poly.mean(), 3))
print("Std  RMSE :", np.round(rmse_cv_poly.std(), 3))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


KeyboardInterrupt: 

In [ ]:
# poly_parameters = {
#     'kernel': ['poly'],
#     'degree': [3, 5, 7, 9],
#     'gamma': ['scale', 'auto'],
#     'coef0': [0, 1, 3],
#     'tol': [1e-3],
#     'C': [0.01, 0.1, 1],
#     'epsilon': [0.01, 0.05, 0.1, 1],
#     'shrinking': [True],
#     'verbose': [False],
#     'max_iter': [-1]
# }


# rbf_parameters = {
#     'kernel': ['rbf'],
#     'gamma': ['scale', 'auto'],
#     'tol': [1e-3],
#     'C': [0.01, 0.1, 1],
#     'epsilon': [0.01, 0.05, 0.1, 0.5, 1],
#     'shrinking': [True],
#     'verbose': [False],
#     'max_iter': [-1]
# }

In [ ]:
# grid_poly = GridSearchCV(
#     estimator=SVR(),
#     param_grid=poly_parameters,
#     cv=5,
#     scoring='neg_root_mean_squared_error', # check others
#     n_jobs=-1,
#     verbose=2
# )

# grid_poly.fit(X_train_scaled, y_train)
# print("Best parameters:", grid_poly.best_params_)
# print("Best RMSE:", abs(grid_poly.best_score_))

# grid_rbf = GridSearchCV(
#     estimator=SVR(),
#     param_grid=rbf_parameters,
#     cv=5,
#     scoring='neg_root_mean_squared_error', # check others
#     n_jobs=-1,
#     verbose=3
# ) 

# grid_rbf.fit(X_train_scaled, y_train)
# print("Best parameters:", grid_rbf.best_params_)
# print("Best RMSE:", abs(grid_rbf.best_score_))

# Partial Poly Graph

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

# Example: 3 feature dataset (X1, X2, X3)
# X is your input matrix with shape (n_samples, n_features)
# y is your target variable
X = np.random.rand(100, 3) * 10  # Random data with 3 features
y = np.sin(X[:, 0]) + 0.5 * X[:, 1] + np.random.randn(100)  # Target variable with noise

# Scaling the data
sc_X = StandardScaler()
sc_y = StandardScaler()

X_scaled = sc_X.fit_transform(X)
y_scaled = sc_y.fit_transform(y.reshape(-1, 1)).ravel()

# Train the SVR model with a polynomial kernel
svr_poly = SVR(kernel='poly', degree=3, C=100, epsilon=0.1)
svr_poly.fit(X_scaled, y_scaled)

# 1. Compute the 95th percentiles for each feature (for input ranges)
percentiles = np.percentile(X, 95, axis=0)

# 2. Plot Partial Poly Graphs for each feature
for feature_index in range(X.shape[1]):
    # Vary the current feature (e.g., X1) across the top 95 percentile range
    feature_range = np.linspace(0, percentiles[feature_index], 1000).reshape(-1, 1)

    # Fix the other features at their mean values
    X_fixed = np.mean(X[:, np.arange(X.shape[1]) != feature_index], axis=0)

    # Create the combined input data with the varied feature and fixed others
    X_combined = np.hstack([feature_range, np.tile(X_fixed, (feature_range.shape[0], 1))])

    # Scale the new data for prediction
    X_combined_scaled = sc_X.transform(X_combined)

    # Predict using the trained SVR model
    y_pred_scaled = svr_poly.predict(X_combined_scaled)

    # Inverse transform the predictions to get original scale
    y_pred = sc_y.inverse_transform(y_pred_scaled)

    # Plot the result
    plt.figure(figsize=(8, 6))
    plt.plot(feature_range, y_pred, label=f"SVR Polynomial Fit (Varying Feature {feature_index+1})")
    plt.title(f'Partial Poly Graph (Varying Feature {feature_index+1})')
    plt.xlabel(f'Feature {feature_index+1}')
    plt.ylabel('Predicted y')
    plt.legend()
    plt.show()

# 3. Plot Partial Dependence for Pairs of Features (2D Contour Plots)
for i in range(X.shape[1]):
    for j in range(i + 1, X.shape[1]):
        # Create a grid of values for Feature i and Feature j
        feature1_range = np.linspace(0, percentiles[i], 100)
        feature2_range = np.linspace(0, percentiles[j], 100)
        
        # Create a meshgrid for the 2D grid of Feature i and Feature j
        feature1_grid, feature2_grid = np.meshgrid(feature1_range, feature2_range)
        
        # Create a combined grid of features for prediction
        X_grid = np.vstack([feature1_grid.ravel(), feature2_grid.ravel()]).T
        
        # Fix the other features at their mean value
        X_fixed = np.mean(X[:, [k for k in range(X.shape[1]) if k != i and k != j]], axis=0)
        
        # Combine the grid values with the fixed feature values
        X_combined = np.hstack([X_grid, np.full((X_grid.shape[0], len(X_fixed)), X_fixed)])

        # Scale the new data for prediction
        X_combined_scaled = sc_X.transform(X_combined)

        # Predict using the trained SVR model
        y_pred_scaled = svr_poly.predict(X_combined_scaled)

        # Inverse transform the predictions to get original scale
        y_pred = sc_y.inverse_transform(y_pred_scaled)

        # Reshape the predictions back into the grid shape
        y_pred_grid = y_pred.reshape(feature1_grid.shape)

        # Plot the results as a contour plot (for 2D data)
        plt.figure(figsize=(8, 6))
        plt.contourf(feature1_range, feature2_range, y_pred_grid, levels=50, cmap='coolwarm')
        plt.colorbar(label='Predicted y')
        plt.title(f'Partial Dependence (Varying Features {i+1} and {j+1})')
        plt.xlabel(f'Feature {i+1}')
        plt.ylabel(f'Feature {j+1}')
        plt.show()


# Finding optimum using model

In [ ]:
# import numpy as np
# import pandas as pd

# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
# from sklearn.svm import SVR
# from sklearn.inspection import permutation_importance

# # --- 1) Columns ---
# PREDICTORS = [
#     'BPS_LAPDISTANCE',
#     'BPS_CURRENTLAPTIMEINMS',
#     'BPS_WORLDPOSITIONX',
#     'BPS_WORLDPOSITIONY'
# ]
# TARGET = 'Target_CURRENTLAPTIMEINMS'

# # --- 2) Data subset & NA drop ---
# use_cols = PREDICTORS + [TARGET]
# data = data[use_cols].dropna().copy()

# X = data[PREDICTORS]
# y = pd.to_numeric(data[TARGET], errors='coerce')

# # --- 3) Train/test split ---
# X_tr, X_te, y_tr, y_te = train_test_split(
#     X, y, test_size=0.2, random_state=42
# )

# --- 4) SVR (RBF kernel) inside a scaling pipeline ---
svr_pipe = make_pipeline(
    StandardScaler(),
    SVR(
        kernel='rbf',
        C=10.0,            # regularisation (tune if needed)
        epsilon=0.1,       # insensitive tube
        gamma='scale'      # default scale is fine
    )
)

# grid_poly.fit(X_train_scaled, y_train_scaled)

# # --- 5) RMSEs ---
# y_fit_tr = svr_pipe.predict(X_tr)
# y_pred_te = svr_pipe.predict(X_te)

# rmse_train = np.sqrt(mean_squared_error(y_tr, y_fit_tr))
# rmse_test  = np.sqrt(mean_squared_error(y_te, y_pred_te))
# print(f"Train RMSE (fitted values): {rmse_train:,.3f}")
# print(f"Test RMSE: {rmse_test:,.3f}")

# --- 6) "Importances": permutation importance on TEST set ---
# perm = permutation_importance(
#     svr_pipe, X_te, y_te,
#     n_repeats=5, random_state=42,
#     scoring='neg_root_mean_squared_error'
# )
# fi = pd.Series(perm.importances_mean, index=PREDICTORS).sort_values(ascending=False)
# print("\nPermutation importances (higher = more degradation when shuffled):")
# print(fi)

# --- 7) Define a realistic search box (stay inside observed data) ---
percentiles = (0.05, 0.95)
bounds = {f: (data[f].quantile(percentiles[0]), data[f].quantile(percentiles[1])) for f in X.columns}

# --- 8) Random search over SVR prediction surface to find min predicted target ---
rng = np.random.default_rng(42)
N = 50_000
candidates = {f: rng.uniform(low=b[0], high=b[1], size=N) for f, b in bounds.items()}
Xcand = pd.DataFrame(candidates)[X.columns]
ycand = grid_poly.predict(Xcand)

imin = int(np.argmin(ycand))
best_combo = Xcand.iloc[imin].to_dict()
best_pred  = ycand[imin]

print("\n=== SVR-suggested first-brake setup (within observed range) ===")
for k, v in best_combo.items():
    print(f"{k}: {v:,.4f}")
print(f"Predicted Target_CURRENTLAPTIMEINMS: {best_pred:,.3f}")

Fitting 5 folds for each of 240 candidates, totalling 1200 fits
[LibSVM]

c:\Users\Haseeb Ijaz\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(



=== SVR-suggested first-brake setup (within observed range) ===
BPS_THROTTLE: 0.7407
BPS_STEER: 0.0067
BPS_BRAKE: 0.0000
BPS_LAPDISTANCE: 269.5061
BPS_WORLDPOSITIONX: 282.3791
BPS_WORLDPOSITIONY: 263.1685
BPS_ext_LAPDISTANCE: 299.9789
BPE_THROTTLE: 0.2812
BPE_STEER: 0.2119
BPE_BRAKE: 0.4872
BPE_LAPDISTANCE: 380.8425
BPE_WORLDPOSITIONX: 373.0243
BPE_WORLDPOSITIONY: 220.7080
BPE_ext_LAPDISTANCE: 395.4552
THS_THROTTLE: 0.5915
THS_STEER: 0.0355
THS_BRAKE: 0.3854
THS_LAPDISTANCE: 168.9022
THS_WORLDPOSITIONX: 222.3611
THS_WORLDPOSITIONY: 349.3771
THS_ext_LAPDISTANCE: 208.7419
THE_THROTTLE: 0.0571
THE_STEER: 0.0666
THE_BRAKE: 0.4583
THE_LAPDISTANCE: 352.8476
THE_WORLDPOSITIONX: 358.9427
THE_WORLDPOSITIONY: 234.4427
THE_ext_LAPDISTANCE: 439.9365
STS_THROTTLE: 0.7775
STS_STEER: 0.0034
STS_BRAKE: 0.7162
STS_LAPDISTANCE: 347.3388
STS_WORLDPOSITIONX: 360.3983
STS_WORLDPOSITIONY: 222.9676
STS_ext_LAPDISTANCE: 352.1718
STM_THROTTLE: 0.4081
STM_STEER: 0.0004
STM_BRAKE: 0.0615
STM_LAPDISTANCE: 413.25

In [6]:
X.columns

Index(['BPS_SPEED', 'BPS_THROTTLE', 'BPS_STEER', 'BPS_BRAKE',
       'BPS_LAPDISTANCE', 'BPS_WORLDPOSITIONX', 'BPS_WORLDPOSITIONY',
       'BPS_ext_LAPDISTANCE', 'BPE_SPEED', 'BPE_THROTTLE', 'BPE_STEER',
       'BPE_BRAKE', 'BPE_LAPDISTANCE', 'BPE_WORLDPOSITIONX',
       'BPE_WORLDPOSITIONY', 'BPE_ext_LAPDISTANCE', 'THS_SPEED',
       'THS_THROTTLE', 'THS_STEER', 'THS_BRAKE', 'THS_LAPDISTANCE',
       'THS_WORLDPOSITIONX', 'THS_WORLDPOSITIONY', 'THS_ext_LAPDISTANCE',
       'THE_SPEED', 'THE_THROTTLE', 'THE_STEER', 'THE_BRAKE',
       'THE_LAPDISTANCE', 'THE_WORLDPOSITIONX', 'THE_WORLDPOSITIONY',
       'THE_ext_LAPDISTANCE', 'STS_SPEED', 'STS_THROTTLE', 'STS_STEER',
       'STS_BRAKE', 'STS_LAPDISTANCE', 'STS_WORLDPOSITIONX',
       'STS_WORLDPOSITIONY', 'STS_ext_LAPDISTANCE', 'STM_SPEED',
       'STM_THROTTLE', 'STM_STEER', 'STM_BRAKE', 'STM_LAPDISTANCE',
       'STM_WORLDPOSITIONX', 'STM_WORLDPOSITIONY', 'STE_SPEED', 'STE_THROTTLE',
       'STE_STEER', 'STE_BRAKE', 'STE_LAPDISTANCE'